In [1]:
from utils.data import (divideLogByFaultTime,
                        msg_node, msg_graph,
                        prepare_repGraph_word2vec,
                        prepare_repGraph_wordfreq,
                        prepare_catGraph_word2vec,
                        RepGraphDataset)
from utils.graph_model import (macro_f1, 
                               Log_Rep_Graph, Log_Rep_GraphAttention, Log_Rep_GraphConv)

# Split log data by fault time

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 读取sel日志数据
sel_log_df = pd.read_csv('./pre_contest/preliminary_sel_log_dataset.csv').drop_duplicates()
# 读取额外的日志数据
#additional_sel_log_df=pd.read_csv('./pre_contest/additional_sel_log_dataset.csv').drop_duplicates()
# 读取训练标签数据：有重复数据！
train_label1 = pd.read_csv('./pre_contest/preliminary_train_label_dataset.csv')
train_label2 = pd.read_csv('./pre_contest/preliminary_train_label_dataset_s.csv')
train_label_df = pd.concat([train_label1,train_label2],axis=0).drop_duplicates()


# 合并日志和标签
sel_log_df['label'] = ''
train_label_df['time'] = train_label_df['fault_time']
train_label_df['msg'] = ''
train_label_df['server_model'] = train_label_df['sn'].map(dict(zip(sel_log_df['sn'],sel_log_df['server_model'])))
train_label_df = train_label_df[['sn', 'time', 'msg', 'server_model', 'label']]

log_label_df = pd.concat([sel_log_df,train_label_df], axis = 0).sort_values(by = 'time')
log_label_df['fault_time'] = ''
log_label_df = log_label_df[['sn', 'fault_time', 'msg', 'time', 'server_model', 'label']]

In [ ]:
FaultTime_log_correspond_label_df, FaultTime_no_label_log_list = divideLogByFaultTime(log_label_df)

In [ ]:
FaultTime_log_correspond_label_df.to_pickle('pre_contest/FaultTime_log_correspond_label_df.new.p')

In [ ]:
FaultTime_log_correspond_label_df.tail()

# Train Word2Vec Model

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
import re

In [ ]:
nodes = []
grp = FaultTime_log_correspond_label_df
grp = grp.loc[~pd.isnull(grp['msg'])]
grp['log_time'] = pd.to_datetime(grp['time'])
grp = grp.sort_values(['log_time'])

for ith in range(len(grp)):
    nodes.append(msg_node(grp['msg'].iloc[ith], 
                          grp['log_time'].iloc[ith], 
                          grp['server_model'].iloc[ith]))

In [ ]:
tokenized_titles = []
for node in nodes:
    if node.msg_type >= 1:
        tokenized_titles.append(word_tokenize(node.attrs[0].lower()))

In [ ]:
tokenized_content = []
for node in nodes:
    if node.msg_type == 0:
        tokenized_content.append(word_tokenize(node.attrs[0].lower()))
    elif (node.msg_type == 1) or (node.msg_type == 2):
        tokenized_content.append(word_tokenize(node.attrs[1].lower()))
    else:
        tokenized_content.append(word_tokenize(' '.join(node.attrs[1:]).lower()))

In [ ]:
tokenized_tot = []
for node in nodes:
    tokenized_tot.append(word_tokenize(' '.join(node.attrs).lower()))

In [ ]:
word_list=[]
for i in range(len(tokenized_titles)):
    xth=[]
    for word in tokenized_titles[i]:
        word_drop=re.sub(r'[^\w]','',str(word)).lower()
        if word_drop:
            xth.append(word_drop)
    word_list.append(xth)

In [ ]:
word_list=[]
for i in range(len(tokenized_content)):
    xth=[]
    for word in tokenized_content[i]:
        word_drop=re.sub(r'[^\w]','',str(word)).lower()
        if word_drop:
            xth.append(word_drop)
    word_list.append(xth)

In [ ]:
word_list=[]
for i in range(len(tokenized_tot)):
    xth=[]
    for word in tokenized_tot[i]:
        word_drop=re.sub(r'[^\w]','',str(word)).lower()
        if word_drop:
            xth.append(word_drop)
    word_list.append(xth)

In [ ]:
word2vec_model = Word2Vec(word_list,vector_size=100, alpha=0.03, window=5, 
                          min_count=1,max_vocab_size=None, sample=1e-3, seed=0, 
                          workers=12, min_alpha=0.0001,sg=1, hs=0, negative=5, 
                          cbow_mean=1, hashfxn=hash, epochs=50, 
                          null_word=0,trim_rule=None, sorted_vocab=1)

In [ ]:
import pickle
file = open('./pre_contest/model_msg_tot_100_log.word2vec.p','wb')
pickle.dump(word2vec_model, file)

In [ ]:
def emb_word2vec(msg_temp, word2vec_model):
    item_raw_word_list = word_tokenize(msg_temp)
    embedding_vector_weighted_sum = []
    for word_xth in item_raw_word_list:
        word_drop_xth = re.sub(r'[^\w]','',str(word)).lower()
        if word_drop_xth:
            vector=word2vec_model.wv[word_drop_xth].reshape(1,-1)[0]
            embedding_vector_weighted_sum.append(np.array(vector))
    return np.mean(np.stack(embedding_vector_weighted_sum),axis=0)

In [ ]:
msg_temp = nodes[0].attrs[1].lower()
emb_word2vec(msg_temp, word2vec_model).shape

# Model training

In [ ]:
import pandas as pd
import numpy as np
import datetime
import networkx as nx
from itertools import chain
from nltk.tokenize import word_tokenize
import pdb
from torchvision import transforms
import torch
from sklearn import metrics
from torch_geometric.loader import DataLoader

In [ ]:
# Load pretrained Doc2Vec Models
import pickle
#gensim_model_title = pickle.load(open('./pre_contest/model_msg_title_20_log.new.p','rb'))
#gensim_model_content = pickle.load(open('./pre_contest/model_msg_content_20_log.new.p','rb'))

v1_word_list=list(pd.read_csv('v1p8/word_frequency_df.txt',sep='\t')['word'])
v1p1_word_list=list(pd.read_csv('v1p8/tags_incomplete.txt',sep='\t',names=['word'])['word'])
v1p2_word_list=list(set(v1_word_list+v1p1_word_list))

# Load dataset & train-test split
import random
log_df = pd.read_pickle('pre_contest/FaultTime_log_correspond_label_df.new.p')
log_df = log_df.loc[~pd.isnull(log_df['msg'])]
log_df['log_time'] = pd.to_datetime(log_df['time'])
log_df = log_df.sort_values(['log_time'])

data_train = []
data_test = []
cnt = 0
for name, grp in log_df.groupby(['sn','fault_time','label']):
    grp['sample_id'] = cnt
    if random.random() >= 0.2:
        data_train.append(grp)
    else:
        data_test.append(grp)
    cnt += 1

In [ ]:
from torchvision import transforms
#t1 = prepare_repGraph(gensim_model_title, gensim_model_content,
#                      dim_title=20, dim_middle=20)
t1 = prepare_repGraph(v1p2_word_list)
trans = transforms.Compose([t1])

trainset = RepGraphDataset(data_train, trans)
testset = RepGraphDataset(data_test, trans)

train_loader = DataLoader(trainset, batch_size=50, shuffle=True)
test_loader = DataLoader(testset, batch_size=100)

In [ ]:
dat = next(iter(train_loader))

In [ ]:
dat.x.shape

In [ ]:
# model setup and training
model = Log_Rep_Graph()
model

In [ ]:
trainer = pl.Trainer(gpus=1, max_epochs=3)

In [ ]:
trainer.fit(model, train_loader, test_loader)

In [ ]:
model.eval()
true_labels = []
pred_labels = []
for dat in test_loader:
    out = model(dat)
    pred_label = np.argmax(out.detach().numpy(),1)
    pred_labels.append(pred_label)
    true_labels.append(dat.label.detach().numpy())

true_vector = np.concatenate(true_labels)
pred_vector = np.concatenate(pred_labels)

cm = metrics.confusion_matrix(true_vector, pred_vector)
print(cm)
macro_f1(true_vector,pred_vector)

# Model training2

In [2]:
import pandas as pd
from torch_geometric.loader import DataLoader
import pytorch_lightning as pl
import numpy as np
from sklearn import metrics

In [3]:
# Load pretrained Doc2Vec Models
import pickle
#gensim_model_title = pickle.load(open('./pre_contest/model_msg_title_20_log.new.p','rb'))
#gensim_model_content = pickle.load(open('./pre_contest/model_msg_content_20_log.new.p','rb'))

#v1_word_list=list(pd.read_csv('v1p8/word_frequency_df.txt',sep='\t')['word'])
#v1p1_word_list=list(pd.read_csv('v1p8/tags_incomplete.txt',sep='\t',names=['word'])['word'])
#v1p2_word_list=list(set(v1_word_list+v1p1_word_list))

gensim_word2vec = pickle.load(open('./pre_contest/model_msg_tot_100_log.word2vec.p','rb'))

# Load dataset & train-test split
import random
log_df = pd.read_pickle('pre_contest/FaultTime_log_correspond_label_df.new.p')
log_df = log_df.loc[~pd.isnull(log_df['msg'])]
log_df['log_time'] = pd.to_datetime(log_df['time'])
log_df = log_df.sort_values(['log_time'])

data_train = []
data_test = []
cnt = 0
for name, grp in log_df.groupby(['sn','fault_time','label']):
    grp['sample_id'] = cnt
    if random.random() >= 0.2:
        data_train.append(grp)
    else:
        data_test.append(grp)
    cnt += 1

In [4]:
from torchvision import transforms
#t1 = prepare_repGraph_word2vec(gensim_word2vec, gensim_word2vec,
#                               dim_title=100, dim_middle=100)
t1 = prepare_catGraph_word2vec(gensim_word2vec, gensim_word2vec,
                               dim_title=100, dim_middle=100)
trans = transforms.Compose([t1])

trainset = RepGraphDataset(data_train, trans)
testset = RepGraphDataset(data_test, trans)

train_loader = DataLoader(trainset, batch_size=50, shuffle=True)
test_loader = DataLoader(testset, batch_size=100)

In [5]:
dat = next(iter(train_loader))
dat

DataBatch(x=[673, 312], label=[50], num_nodes=673, edge_index=[2, 1225], edge_attr=[1225], id=[50], server_model=[50], batch=[673], ptr=[51])

In [6]:
# model setup and training
model = Log_Rep_Graph(input_dim=312, hidden_dim=96, learning_rate=1e-3)
model

Log_Rep_Graph(
  (conv1): GCNConv(312, 96)
  (pool1): TopKPooling(96, ratio=0.5, multiplier=1.0)
  (conv2): GCNConv(96, 4)
  (logsoftmax_func): LogSoftmax(dim=1)
)

In [9]:
model = Log_Rep_GraphAttention(input_dim=312, hidden_dim=96, num_head=3, learning_rate=1e-3)
model

Log_Rep_GraphAttention(
  (conv1): GATConv(312, 96, heads=3)
  (pool1): TopKPooling(288, ratio=0.5, multiplier=1.0)
  (conv2): GCNConv(288, 4)
  (logsoftmax_func): LogSoftmax(dim=1)
)

In [13]:
model = Log_Rep_GraphConv(input_dim=312, hidden_dim=96, learning_rate=1e-3)
model

Log_Rep_GraphConv(
  (conv1): GraphConv(312, 96)
  (pool1): TopKPooling(96, ratio=0.5, multiplier=1.0)
  (conv2): GCNConv(96, 4)
  (logsoftmax_func): LogSoftmax(dim=1)
)

In [ ]:
model.lr=1e-4

In [14]:
trainer = pl.Trainer(gpus=1, max_epochs=10)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [15]:
trainer.fit(model, train_loader, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name            | Type        | Params
------------------------------------------------
0 | conv1           | GraphConv   | 60.0 K
1 | pool1           | TopKPooling | 96    
2 | conv2           | GCNConv     | 388   
3 | logsoftmax_func | LogSoftmax  | 0     
------------------------------------------------
60.5 K    Trainable params
0         Non-trainable params
60.5 K    Total params
0.242     Total estimated model params size (MB)


/home/qingyang/miniconda3/envs/ares_new/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/qingyang/miniconda3/envs/ares_new/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


In [16]:
model.eval()
true_labels = []
pred_labels = []
for dat in test_loader:
    out = model(dat)
    pred_label = np.argmax(out.detach().numpy(),1)
    pred_labels.append(pred_label)
    true_labels.append(dat.label.detach().numpy())

true_vector = np.concatenate(true_labels)
pred_vector = np.concatenate(pred_labels)

cm = metrics.confusion_matrix(true_vector, pred_vector)
print(cm)
macro_f1(true_vector,pred_vector)

[[   0  200   66    8]
 [   0  490  140   40]
 [   0   45 1783   80]
 [   0    8  166  325]]
Task 1:
 Prcesion 0.00, Recall 0.00, F1 0.00
Task 2:
 Prcesion 0.66, Recall 0.73, F1 0.69
Task 3:
 Prcesion 0.83, Recall 0.93, F1 0.88
Task 4:
 Prcesion 0.72, Recall 0.65, F1 0.68


0.42108132951054617